In [202]:
import pandas as pd 
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.io as pio
import seaborn as sns
import numpy as np
pd.set_option('display.max_columns', 700)
import tensorflow as tf

import warnings
warnings.filterwarnings("ignore") 

In [203]:
df_train = pd.read_csv('TRAIN (1).csv')
df_train['order_date'] = pd.to_datetime(df_train['order_date'], format = '%m/%d/%y')
df_train['evsd'] = pd.to_datetime(df_train['evsd'], format = '%m/%d/%y')
df_train['tag'] = 'train'

print(df_train.shape)
df_train.head()

(259287, 14)


,region_id,warehouse_id,d_id,isbn_id,order_id,order_date,evsd,po_source_ind,quantity_ordered,quantity_submitted,quantity_received,row_group_ind,visibility,tag
0,1,JFK2,540,30718,67271,2022-11-02,2022-11-14,AFT Lite,54,54,54,4,9,train
1,1,JFK2,540,30718,125389,2022-11-20,2022-12-01,AFT Lite,60,60,60,5,3,train
2,1,JFK2,648,16750,11165,2022-09-30,2022-10-03,AFT Lite,6,6,6,1,2,train
3,1,JFK2,648,16750,27008,2022-10-14,2022-10-17,AFT Lite,6,6,6,2,2,train
4,1,JFK2,7,8748,33598,2022-11-15,2022-11-16,AFT Lite,8,8,8,1,1,train


In [204]:
# Create a new column that combines the values of the existing columns
df_train['unique_id'] = df_train.apply(lambda row: f"{row['warehouse_id']}_{row['d_id']}_{row['isbn_id']}_{row['order_id']}", axis=1)

# Drop duplicates based on the new column
df_unique = df_train.drop_duplicates(subset=['unique_id'])

df_unique.shape, df_train.shape

((101777, 15), (259287, 15))

In [205]:
df_train.head()

,region_id,warehouse_id,d_id,isbn_id,order_id,order_date,evsd,po_source_ind,quantity_ordered,quantity_submitted,quantity_received,row_group_ind,visibility,tag,unique_id
0,1,JFK2,540,30718,67271,2022-11-02,2022-11-14,AFT Lite,54,54,54,4,9,train,JFK2_540_30718_67271
1,1,JFK2,540,30718,125389,2022-11-20,2022-12-01,AFT Lite,60,60,60,5,3,train,JFK2_540_30718_125389
2,1,JFK2,648,16750,11165,2022-09-30,2022-10-03,AFT Lite,6,6,6,1,2,train,JFK2_648_16750_11165
3,1,JFK2,648,16750,27008,2022-10-14,2022-10-17,AFT Lite,6,6,6,2,2,train,JFK2_648_16750_27008
4,1,JFK2,7,8748,33598,2022-11-15,2022-11-16,AFT Lite,8,8,8,1,1,train,JFK2_7_8748_33598


In [206]:
df_train['visibility'].dtype

dtype('int64')

In [207]:
# How long does order usually take to deliver?
# df_unique['order_delivery_days'] = (df_unique['evsd'] - df_unique['order_date']).dt.days
# df_test['order_delivery_days'] = (df_test['evsd'] - df_test['order_date']).dt.days

df_unique['consider'] =   np.where((df_unique['evsd'] - df_unique['order_date']).dt.days  == df_unique['visibility'], True, False)




In [208]:
df_unique['consider'].value_counts(dropna= False)

consider
True     54085
False    47692
Name: count, dtype: int64

In [209]:
df_test = pd.read_csv('Test.csv', encoding = 'latin1')
# df_test.columns = ['LOCATION', 'd_id', 'isbn_id', 'order_id_1', 'order_date', 'evsd',
#        'row_group_ind', 'visibility', 'quantity_ordered', 'quantity_submitted',
#        'quantity_received']
df_test['order_date'] = pd.to_datetime(df_test['order_date'], format = '%m/%d/%y')
df_test['evsd'] = pd.to_datetime(df_test['evsd'], format = '%m/%d/%y')
df_test['tag'] = 'test'
print(df_test.shape)
df_test.head()

(98309, 14)


,region_id,warehouse_id,d_id,isbn_id,order_id,order_date,evsd,po_source_ind,quantity_ordered,quantity_submitted,quantity_received,row_group_ind,visibility,tag
0,1,JFK2,684,674,45573,2022-12-15,2022-12-19,AFT Lite,42,42,42,3,3,test
1,1,JFK2,1,19822,110331,2022-12-27,2022-12-30,AFT Lite,6,6,6,0,3,test
2,1,JFK2,509,19749,119943,2022-12-15,2022-12-17,AFT Lite,12,12,12,3,1,test
3,1,JFK2,584,19747,43091,2022-12-20,2022-12-23,AFT Lite,12,12,12,4,3,test
4,1,JFK2,540,20260,117854,2022-12-26,2023-01-04,AFT Lite,60,60,60,1,8,test


In [210]:
# Apply same filter on Test Data too
# Create a new column that combines the values of the existing columns
df_test['unique_id'] = df_test.apply(lambda row: f"{row['warehouse_id']}_{row['d_id']}_{row['isbn_id']}_{row['order_id']}", axis=1)
print(df_test.shape)
# Drop duplicates based on the new column
df_test = df_test.drop_duplicates(subset=['unique_id'])

df_test.shape

(98309, 15)


(40996, 15)

In [211]:
df_test['consider'] =   np.where((df_test['evsd'] - df_test['order_date']).dt.days  == df_test['visibility'], True, False)
df_test['consider'].value_counts(dropna= False)

consider
True     22406
False    18590
Name: count, dtype: int64

## Final DATA : ``df_unique`` and ``df_test``

In [212]:
df_unique = df_unique.loc[df_unique['consider'] == True].reset_index(drop=True)
df_test = df_test.loc[df_test['consider'] == True].reset_index(drop=True)


In [213]:
df_test.shape

(22406, 16)

In [214]:
# Feature Engineering
for col in ['d_id', 'isbn_id', 'order_id']:
  df_unique[col] = df_unique[col].astype(int).astype(str)
  df_test[col] = df_test[col].astype(int).astype(str)
  # df_test_n[col] = df_test_n[col].astype(int).astype(str)

print(df_unique.shape)
display(df_unique.head(2))

print(df_test.shape)
df_test.head(2)

(54085, 16)


,region_id,warehouse_id,d_id,isbn_id,order_id,order_date,evsd,po_source_ind,quantity_ordered,quantity_submitted,quantity_received,row_group_ind,visibility,tag,unique_id,consider
0,1,JFK2,7,8748,33598,2022-11-15,2022-11-16,AFT Lite,8,8,8,1,1,train,JFK2_7_8748_33598,True
1,1,JFK2,7,8748,142228,2022-11-01,2022-11-02,AFT Lite,16,16,16,3,1,train,JFK2_7_8748_142228,True


(22406, 16)


,region_id,warehouse_id,d_id,isbn_id,order_id,order_date,evsd,po_source_ind,quantity_ordered,quantity_submitted,quantity_received,row_group_ind,visibility,tag,unique_id,consider
0,1,JFK2,1,19822,110331,2022-12-27,2022-12-30,AFT Lite,6,6,6,0,3,test,JFK2_1_19822_110331,True
1,1,JFK2,584,19747,43091,2022-12-20,2022-12-23,AFT Lite,12,12,12,4,3,test,JFK2_584_19747_43091,True


In [215]:

df_unique['order_day'] = df_unique['order_date'].dt.day
df_unique['order_dayofweek'] = df_unique['order_date'].dt.dayofweek
df_unique['order_dayofyear'] = df_unique['order_date'].dt.dayofyear
df_unique['order_month'] = df_unique['order_date'].dt.month

df_unique['delivered_day'] = df_unique['evsd'].dt.day
df_unique['delivered_dayofweek'] = df_unique['evsd'].dt.dayofweek
df_unique['delivered_dayofyear'] = df_unique['evsd'].dt.dayofyear
df_unique['delivered_month'] = df_unique['evsd'].dt.month

df_unique['order_month_sin'] = np.sin(2 * np.pi * df_unique['order_month'] / 12)
df_unique['order_month_cos'] = np.cos(2 * np.pi * df_unique['order_month'] / 12)

df_unique['order_dayofweek_sin'] = np.sin(2 * np.pi * df_unique['order_dayofweek'] / 7)
df_unique['order_dayofweek_cos'] = np.cos(2 * np.pi * df_unique['order_dayofweek'] / 7)

df_unique['delivered_month_sin'] = np.sin(2 * np.pi * df_unique['delivered_month'] / 12)
df_unique['delivered_month_cos'] = np.cos(2 * np.pi * df_unique['delivered_month'] / 12)

df_unique['delivered_dayofweek_sin'] = np.sin(2 * np.pi * df_unique['delivered_dayofweek'] / 7)
df_unique['delivered_dayofweek_cos'] = np.cos(2 * np.pi * df_unique['delivered_dayofweek'] / 7)

df_unique.drop(['order_month','delivered_month'], axis =1, inplace = True)
display(df_unique.head())


df_test['order_day'] = df_test['order_date'].dt.day
df_test['order_dayofweek'] = df_test['order_date'].dt.dayofweek
df_test['order_dayofyear'] = df_test['order_date'].dt.dayofyear
df_test['order_month'] = df_test['order_date'].dt.month

df_test['delivered_day'] = df_test['evsd'].dt.day
df_test['delivered_dayofweek'] = df_test['evsd'].dt.dayofweek
df_test['delivered_dayofyear'] = df_test['evsd'].dt.dayofyear
df_test['delivered_month'] = df_test['evsd'].dt.month

df_test['order_month_sin'] = np.sin(2 * np.pi * df_test['order_month'] / 12)
df_test['order_month_cos'] = np.cos(2 * np.pi * df_test['order_month'] / 12)

df_test['order_dayofweek_sin'] = np.sin(2 * np.pi * df_test['order_dayofweek'] / 7)
df_test['order_dayofweek_cos'] = np.cos(2 * np.pi * df_test['order_dayofweek'] / 7)

df_test['delivered_month_sin'] = np.sin(2 * np.pi * df_test['delivered_month'] / 12)
df_test['delivered_month_cos'] = np.cos(2 * np.pi * df_test['delivered_month'] / 12)

df_test['delivered_dayofweek_sin'] = np.sin(2 * np.pi * df_test['delivered_dayofweek'] / 7)
df_test['delivered_dayofweek_cos'] = np.cos(2 * np.pi * df_test['delivered_dayofweek'] / 7)

df_test.drop(['order_month','delivered_month'], axis =1, inplace = True)
display(df_test.head())


# df_test_n['order_day'] = df_test_n['order_date'].dt.day
# df_test_n['order_dayofweek'] = df_test_n['order_date'].dt.dayofweek
# df_test_n['order_dayofyear'] = df_test_n['order_date'].dt.dayofyear
# df_test_n['order_month'] = df_test_n['order_date'].dt.month

# df_test_n['delivered_day'] = df_test_n['evsd'].dt.day
# df_test_n['delivered_dayofweek'] = df_test_n['evsd'].dt.dayofweek
# df_test_n['delivered_dayofyear'] = df_test_n['evsd'].dt.dayofyear
# df_test_n['delivered_month'] = df_test_n['evsd'].dt.month

# df_test_n['order_month_sin'] = np.sin(2 * np.pi * df_test_n['order_month'] / 12)
# df_test_n['order_month_cos'] = np.cos(2 * np.pi * df_test_n['order_month'] / 12)

# df_test_n['order_dayofweek_sin'] = np.sin(2 * np.pi * df_test_n['order_dayofweek'] / 7)
# df_test_n['order_dayofweek_cos'] = np.cos(2 * np.pi * df_test_n['order_dayofweek'] / 7)

# df_test_n['delivered_month_sin'] = np.sin(2 * np.pi * df_test_n['delivered_month'] / 12)
# df_test_n['delivered_month_cos'] = np.cos(2 * np.pi * df_test_n['delivered_month'] / 12)

# df_test_n['delivered_dayofweek_sin'] = np.sin(2 * np.pi * df_test_n['delivered_dayofweek'] / 7)
# df_test_n['delivered_dayofweek_cos'] = np.cos(2 * np.pi * df_test_n['delivered_dayofweek'] / 7)

# df_test_n.drop(['order_month','delivered_month'], axis =1, inplace = True)
# display(df_test_n.head())


,region_id,warehouse_id,d_id,isbn_id,order_id,order_date,evsd,po_source_ind,quantity_ordered,quantity_submitted,quantity_received,row_group_ind,visibility,tag,unique_id,consider,order_day,order_dayofweek,order_dayofyear,delivered_day,delivered_dayofweek,delivered_dayofyear,order_month_sin,order_month_cos,order_dayofweek_sin,order_dayofweek_cos,delivered_month_sin,delivered_month_cos,delivered_dayofweek_sin,delivered_dayofweek_cos
0,1,JFK2,7,8748,33598,2022-11-15,2022-11-16,AFT Lite,8,8,8,1,1,train,JFK2_7_8748_33598,True,15,1,319,16,2,320,-0.500000,0.866025,0.781831,0.623490,-0.500000,0.866025,0.974928,-0.222521
1,1,JFK2,7,8748,142228,2022-11-01,2022-11-02,AFT Lite,16,16,16,3,1,train,JFK2_7_8748_142228,True,1,1,305,2,2,306,-0.500000,0.866025,0.781831,0.623490,-0.500000,0.866025,0.974928,-0.222521
2,1,JFK2,7,8748,27330,2022-11-05,2022-11-06,AFT Lite,0,24,0,6,1,train,JFK2_7_8748_27330,True,5,5,309,6,6,310,-0.500000,0.866025,-0.974928,-0.222521,-0.500000,0.866025,-0.781831,0.623490
3,1,JFK2,7,17780,122956,2022-11-12,2022-11-13,AFT Lite,36,36,36,3,1,train,JFK2_7_17780_122956,True,12,5,316,13,6,317,-0.500000,0.866025,-0.974928,-0.222521,-0.500000,0.866025,-0.781831,0.623490
4,1,JFK2,7,27826,175768,2022-10-11,2022-10-12,AFT Lite,24,24,24,5,1,train,JFK2_7_27826_175768,True,11,1,284,12,2,285,-0.866025,0.500000,0.781831,0.623490,-0.866025,0.500000,0.974928,-0.222521


,region_id,warehouse_id,d_id,isbn_id,order_id,order_date,evsd,po_source_ind,quantity_ordered,quantity_submitted,quantity_received,row_group_ind,visibility,tag,unique_id,consider,order_day,order_dayofweek,order_dayofyear,delivered_day,delivered_dayofweek,delivered_dayofyear,order_month_sin,order_month_cos,order_dayofweek_sin,order_dayofweek_cos,delivered_month_sin,delivered_month_cos,delivered_dayofweek_sin,delivered_dayofweek_cos
0,1,JFK2,1,19822,110331,2022-12-27,2022-12-30,AFT Lite,6,6,6,0,3,test,JFK2_1_19822_110331,True,27,1,361,30,4,364,-2.449294e-16,1.0,0.781831,0.623490,-2.449294e-16,1.0,-0.433884,-0.900969
1,1,JFK2,584,19747,43091,2022-12-20,2022-12-23,AFT Lite,12,12,12,4,3,test,JFK2_584_19747_43091,True,20,1,354,23,4,357,-2.449294e-16,1.0,0.781831,0.623490,-2.449294e-16,1.0,-0.433884,-0.900969
2,1,JFK2,540,32203,112020,2022-12-07,2022-12-16,AFT Lite,18,18,18,5,9,test,JFK2_540_32203_112020,True,7,2,341,16,4,350,-2.449294e-16,1.0,0.974928,-0.222521,-2.449294e-16,1.0,-0.433884,-0.900969
3,1,JFK2,509,10609,26126,2022-12-08,2022-12-10,AFT Lite,18,18,18,6,2,test,JFK2_509_10609_26126,True,8,3,342,10,5,344,-2.449294e-16,1.0,0.433884,-0.900969,-2.449294e-16,1.0,-0.974928,-0.222521
4,1,JFK2,509,8427,26144,2022-12-29,2022-12-31,AFT Lite,0,36,0,1,2,test,JFK2_509_8427_26144,True,29,3,363,31,5,365,-2.449294e-16,1.0,0.433884,-0.900969,-2.449294e-16,1.0,-0.974928,-0.222521


In [216]:
# Define a funtion to peform one hot encoding for the top 10 most frequent categories of Categorical Features
def one_hot_top10(df,feature,top10_categories):
    for category in top10_categories:
        df[feature+'_'+category]=np.where(df[feature]==category,1,0)

In [217]:
df_unique['row_group_ind'] = df_unique['row_group_ind'].astype(str)
df_test['row_group_ind'] = df_test['row_group_ind'].astype(str)
df_unique['order_dayofweek'] = df_unique['order_dayofweek'].astype(str)
df_test['order_dayofweek'] = df_test['order_dayofweek'].astype(str)
df_unique['delivered_dayofweek'] = df_unique['delivered_dayofweek'].astype(str)
df_test['delivered_dayofweek'] = df_test['delivered_dayofweek'].astype(str)
# df_test_n['row_group_ind'] = df_test_n['row_group_ind'].astype(str)

var_dict_train_ ={}

for col in ['row_group_ind', 'd_id', 'order_dayofweek', 'delivered_dayofweek']:
  var_dict_train_[col] = [x for x in df_unique[col].value_counts().sort_values(ascending=False).index]


display(var_dict_train_)

{'row_group_ind': ['3', '4', '2', '5', '1', '6', '7', '0'],
 'd_id': ['509',
  '7',
  '110',
  '584',
  '648',
  '566',
  '1',
  '643',
  '454',
  '639',
  '684',
  '410',
  '540',
  '174',
  '472',
  '763',
  '830',
  '136',
  '711',
  '343',
  '524',
  '652',
  '18',
  '373',
  '13',
  '393',
  '845',
  '218',
  '302',
  '241',
  '127',
  '613'],
 'order_dayofweek': ['6', '1', '0', '3', '2', '4', '5'],
 'delivered_dayofweek': ['1', '5', '2', '4', '3', '0', '6']}

In [218]:
for key,val in var_dict_train_.items():
  one_hot_top10(df_unique,key,val)

for key,val in var_dict_train_.items():
  one_hot_top10(df_test,key,val)

# for key,val in var_dict_train_.items():
#   one_hot_top10(df_test_n,key,val)

display(df_unique.head())
display(df_test.head())

,region_id,warehouse_id,d_id,isbn_id,order_id,order_date,evsd,po_source_ind,quantity_ordered,quantity_submitted,quantity_received,row_group_ind,visibility,tag,unique_id,consider,order_day,order_dayofweek,order_dayofyear,delivered_day,delivered_dayofweek,delivered_dayofyear,order_month_sin,order_month_cos,order_dayofweek_sin,order_dayofweek_cos,delivered_month_sin,delivered_month_cos,delivered_dayofweek_sin,delivered_dayofweek_cos,row_group_ind_3,row_group_ind_4,row_group_ind_2,row_group_ind_5,row_group_ind_1,row_group_ind_6,row_group_ind_7,row_group_ind_0,d_id_509,d_id_7,d_id_110,d_id_584,d_id_648,d_id_566,d_id_1,d_id_643,d_id_454,d_id_639,d_id_684,d_id_410,d_id_540,d_id_174,d_id_472,d_id_763,d_id_830,d_id_136,d_id_711,d_id_343,d_id_524,d_id_652,d_id_18,d_id_373,d_id_13,d_id_393,d_id_845,d_id_218,d_id_302,d_id_241,d_id_127,d_id_613,order_dayofweek_6,order_dayofweek_1,order_dayofweek_0,order_dayofweek_3,order_dayofweek_2,order_dayofweek_4,order_dayofweek_5,delivered_dayofweek_1,delivered_dayofweek_5,delivered_dayofweek_2,delivered_dayofweek_4,delivered_dayofweek_3,delivered_dayofweek_0,delivered_dayofweek_6
0,1,JFK2,7,8748,33598,2022-11-15,2022-11-16,AFT Lite,8,8,8,1,1,train,JFK2_7_8748_33598,True,15,1,319,16,2,320,-0.500000,0.866025,0.781831,0.623490,-0.500000,0.866025,0.974928,-0.222521,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0
1,1,JFK2,7,8748,142228,2022-11-01,2022-11-02,AFT Lite,16,16,16,3,1,train,JFK2_7_8748_142228,True,1,1,305,2,2,306,-0.500000,0.866025,0.781831,0.623490,-0.500000,0.866025,0.974928,-0.222521,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0
2,1,JFK2,7,8748,27330,2022-11-05,2022-11-06,AFT Lite,0,24,0,6,1,train,JFK2_7_8748_27330,True,5,5,309,6,6,310,-0.500000,0.866025,-0.974928,-0.222521,-0.500000,0.866025,-0.781831,0.623490,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1
3,1,JFK2,7,17780,122956,2022-11-12,2022-11-13,AFT Lite,36,36,36,3,1,train,JFK2_7_17780_122956,True,12,5,316,13,6,317,-0.500000,0.866025,-0.974928,-0.222521,-0.500000,0.866025,-0.781831,0.623490,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1
4,1,JFK2,7,27826,175768,2022-10-11,2022-10-12,AFT Lite,24,24,24,5,1,train,JFK2_7_27826_175768,True,11,1,284,12,2,285,-0.866025,0.500000,0.781831,0.623490,-0.866025,0.500000,0.974928,-0.222521,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0


,region_id,warehouse_id,d_id,isbn_id,order_id,order_date,evsd,po_source_ind,quantity_ordered,quantity_submitted,quantity_received,row_group_ind,visibility,tag,unique_id,consider,order_day,order_dayofweek,order_dayofyear,delivered_day,delivered_dayofweek,delivered_dayofyear,order_month_sin,order_month_cos,order_dayofweek_sin,order_dayofweek_cos,delivered_month_sin,delivered_month_cos,delivered_dayofweek_sin,delivered_dayofweek_cos,row_group_ind_3,row_group_ind_4,row_group_ind_2,row_group_ind_5,row_group_ind_1,row_group_ind_6,row_group_ind_7,row_group_ind_0,d_id_509,d_id_7,d_id_110,d_id_584,d_id_648,d_id_566,d_id_1,d_id_643,d_id_454,d_id_639,d_id_684,d_id_410,d_id_540,d_id_174,d_id_472,d_id_763,d_id_830,d_id_136,d_id_711,d_id_343,d_id_524,d_id_652,d_id_18,d_id_373,d_id_13,d_id_393,d_id_845,d_id_218,d_id_302,d_id_241,d_id_127,d_id_613,order_dayofweek_6,order_dayofweek_1,order_dayofweek_0,order_dayofweek_3,order_dayofweek_2,order_dayofweek_4,order_dayofweek_5,delivered_dayofweek_1,delivered_dayofweek_5,delivered_dayofweek_2,delivered_dayofweek_4,delivered_dayofweek_3,delivered_dayofweek_0,delivered_dayofweek_6
0,1,JFK2,1,19822,110331,2022-12-27,2022-12-30,AFT Lite,6,6,6,0,3,test,JFK2_1_19822_110331,True,27,1,361,30,4,364,-2.449294e-16,1.0,0.781831,0.623490,-2.449294e-16,1.0,-0.433884,-0.900969,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0
1,1,JFK2,584,19747,43091,2022-12-20,2022-12-23,AFT Lite,12,12,12,4,3,test,JFK2_584_19747_43091,True,20,1,354,23,4,357,-2.449294e-16,1.0,0.781831,0.623490,-2.449294e-16,1.0,-0.433884,-0.900969,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0
2,1,JFK2,540,32203,112020,2022-12-07,2022-12-16,AFT Lite,18,18,18,5,9,test,JFK2_540_32203_112020,True,7,2,341,16,4,350,-2.449294e-16,1.0,0.974928,-0.222521,-2.449294e-16,1.0,-0.433884,-0.900969,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0
3,1,JFK2,509,10609,26126,2022-12-08,2022-12-10,AFT Lite,18,18,18,6,2,test,JFK2_509_10609_26126,True,8,3,342,10,5,344,-2.449294e-16,1.0,0.433884,-0.900969,-2.449294e-16,1.0,-0.974928,-0.222521,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0
4,1,JFK2,509,8427,26144,2022-12-29,2022-12-31,AFT Lite,0,36,0,1,2,test,JFK2_509_8427_26144,True,29,3,363,31,5,365,-2.449294e-16,1.0,0.433884,-0.900969,-2.449294e-16,1.0,-0.974928,-0.222521,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0


In [219]:
df_grouped = df_unique[['d_id', 'evsd']].groupby('d_id').nunique()

# Filter d_id with more than 10 time periods
ids = list(df_grouped[df_grouped['evsd']>=10].index)
df_unique = df_unique[df_unique['d_id'].isin(ids)].reset_index(drop = True).copy()


df_grouped_t = df_test[['d_id', 'evsd']].groupby('d_id').nunique()
# Filter d_id with more than 10 time periods
ids_t = list(df_grouped_t[df_grouped_t['evsd']>=10].index)
df_test = df_test[df_test['d_id'].isin(ids_t)].reset_index(drop = True).copy()


In [220]:
df = pd.concat([df_unique, df_test]).reset_index(drop = True)

In [221]:

vc_cols = ['delivered_day', 'delivered_dayofweek','order_day', 'order_dayofweek']
for col in vc_cols:
    temp_dict = df_unique[col].value_counts(normalize=True).to_dict()
    df[f"{col}_VC"] = df[col].map(temp_dict)
# for col in vc_cols:
#     temp_dict = df_unique[col].value_counts(normalize=True).to_dict()
#     df_unique[f"{col}_VC"] = df_unique[col].map(temp_dict)
#     df_test[f"{col}_VC"] = df_test[col].map(temp_dict)
    # df_test_n[f"{col}_VC"] = df_test_n[col].map(temp_dict)




In [222]:
req_cols = [col for col in df.columns if col not in ['region_id', 'warehouse_id', 'd_id', 'isbn_id',
                                                            'order_month_cos', 'order_month_sin','order_id',
                                                              'order_date', 'evsd', 'po_source_ind', 'tag',
                                                                'unique_id', 'row_group_ind','quantity_received', 
                                                                'consider', 'order_dayofyear', 'delivered_dayofyear',
                                                                'delivered_day', 'delivered_dayofweek','order_day', 'order_dayofweek', 
                                                                'quantity_received_scaled', 
                                                                'order_dayofweek_sin', 'order_dayofweek_cos', 
                                                                'delivered_month_sin', 'delivered_month_cos', 
 'delivered_dayofweek_sin', 'delivered_dayofweek_cos']]


In [223]:
def mae(y_true, predictions):
    y_true, predictions = np.array(y_true), np.array(predictions)
    return np.mean(np.abs(y_true - predictions))

def new_metric(df):
    df['mae'] = df[['quantity_received', 'preds']].apply(lambda row: mae(row['quantity_received'], row['preds']), axis=1)
    final_error = df['mae'].sum()/ df['preds'].sum()
    return final_error

In [224]:
# StandardScaler on Target Varianle and quantity columns

from sklearn import preprocessing


# scaler_t = preprocessing.MinMaxScaler()
# df_unique['quantity_received'] = scaler_t.fit_transform(df_unique['quantity_received'].values.reshape(-1, 1))
# df_test['quantity_received'] = scaler_t.fit_transform(df_test['quantity_received'].values.reshape(-1, 1))
# df_test_n['quantity_received'] = scaler_t.transform(df_test_n['quantity_received'].values.reshape(-1, 1))

# y = df_unique['quantity_received']
# y_mean = np.mean(y)
# y_std = np.std(y)

# df_unique['quantity_received_scaled'] = (y - y_mean)/y_std


# y_test = df_test['quantity_received']
# y_mean_t = np.mean(y_test)
# y_std_t = np.std(y_test)

# df_test['quantity_received_scaled'] = (y_test - y_mean_t)/y_std_t


# scaler_s = preprocessing.QuantileTransformer()
# df_unique['quantity_ordered'] = scaler_s.fit_transform(df_unique['quantity_ordered'].values.reshape(-1, 1))
# df_unique['quantity_submitted'] = scaler_s.fit_transform(df_unique['quantity_submitted'].values.reshape(-1, 1))
# df_unique['visibility'] = scaler_s.fit_transform(df_unique['visibility'].values.reshape(-1, 1))
# df_test['quantity_ordered'] = scaler_s.transform(df_test['quantity_ordered'].values.reshape(-1, 1))
# # df_test_n['quantity_ordered'] = scaler_s.transform(df_test_n['quantity_ordered'].values.reshape(-1, 1))
# df_test['quantity_submitted'] = scaler_s.transform(df_test['quantity_submitted'].values.reshape(-1, 1))
# df_test['visibility'] = scaler_s.transform(df_test['visibility'].values.reshape(-1, 1))
# df_test_n['quantity_submitted'] = scaler_s.transform(df_test_n['quantity_submitted'].values.reshape(-1, 1))


In [225]:
# from sklearn.preprocessing import QuantileTransformer
from sklearn.metrics import mean_absolute_error, r2_score
import xgboost as xgb





In [228]:

from sklearn.model_selection import train_test_split

In [178]:



# scaler_t = preprocessing.MinMaxScaler()
# y = df['quantity_received']
# y_mean = np.mean(y)
# y_std = np.std(y)

# y_scaled = (y - y_mean)/y_std


# import numpy as np

# def minmax_scaler(data):
#     min_val = np.min(data)
#     max_val = np.max(data)
#     scaled = (data - min_val) / (max_val - min_val)
#     return scaled

# y_scaled = minmax_scaler(y)



# X = df[req_cols].copy()
# # y = df[['quantity_received']].copy()
# X_train, X_test = train_test_split(X, test_size=0.3, random_state=42, shuffle=True, stratify=df[['d_id']])
# y_train, y_test = train_test_split(y_scaled, test_size=0.3, random_state=42, shuffle=True, stratify=df[['d_id']])

In [179]:
# def minmax_scaler(data):
#     min_val = np.min(data)
#     max_val = np.max(data)
#     scaled = (data - min_val) / (max_val - min_val)
#     return scaled, min_val, max_val

# def inverse_minmax_scaler(scaled_data, min_val, max_val):
#     original = scaled_data * (max_val - min_val) + min_val
#     return original



In [229]:
req_cols

['quantity_ordered',
 'quantity_submitted',
 'visibility',
 'row_group_ind_3',
 'row_group_ind_4',
 'row_group_ind_2',
 'row_group_ind_5',
 'row_group_ind_1',
 'row_group_ind_6',
 'row_group_ind_7',
 'row_group_ind_0',
 'd_id_509',
 'd_id_7',
 'd_id_110',
 'd_id_584',
 'd_id_648',
 'd_id_566',
 'd_id_1',
 'd_id_643',
 'd_id_454',
 'd_id_639',
 'd_id_684',
 'd_id_410',
 'd_id_540',
 'd_id_174',
 'd_id_472',
 'd_id_763',
 'd_id_830',
 'd_id_136',
 'd_id_711',
 'd_id_343',
 'd_id_524',
 'd_id_652',
 'd_id_18',
 'd_id_373',
 'd_id_13',
 'd_id_393',
 'd_id_845',
 'd_id_218',
 'd_id_302',
 'd_id_241',
 'd_id_127',
 'd_id_613',
 'order_dayofweek_6',
 'order_dayofweek_1',
 'order_dayofweek_0',
 'order_dayofweek_3',
 'order_dayofweek_2',
 'order_dayofweek_4',
 'order_dayofweek_5',
 'delivered_dayofweek_1',
 'delivered_dayofweek_5',
 'delivered_dayofweek_2',
 'delivered_dayofweek_4',
 'delivered_dayofweek_3',
 'delivered_dayofweek_0',
 'delivered_dayofweek_6',
 'delivered_day_VC',
 'delivered_da

In [230]:
used_cols = ['quantity_ordered',
 'quantity_submitted',
 'visibility',
 'row_group_ind_3',
 'row_group_ind_4',
 'row_group_ind_2',
 'row_group_ind_5',
 'row_group_ind_1',
 'row_group_ind_6',
 'row_group_ind_7',
 'row_group_ind_0','order_dayofweek_6',
 'order_dayofweek_1',
 'order_dayofweek_0',
 'order_dayofweek_3',
 'order_dayofweek_2',
 'order_dayofweek_4',
 'order_dayofweek_5',
 'delivered_dayofweek_1',
 'delivered_dayofweek_5',
 'delivered_dayofweek_2',
 'delivered_dayofweek_4',
 'delivered_dayofweek_3',
 'delivered_dayofweek_0',
 'delivered_dayofweek_6',
 'delivered_day_VC',
 'delivered_dayofweek_VC',
 'order_day_VC',
 'order_dayofweek_VC']

In [233]:
import numpy as np
import tensorflow as tf
from sklearn.metrics import r2_score, mean_absolute_error

# Define min-max scaler
def minmax_scaler(data, min_val=None, max_val=None):
    if min_val is None:
        min_val = np.min(data)
    if max_val is None:
        max_val = np.max(data)
    scaled = (data - min_val) / (max_val - min_val)
    return scaled, min_val, max_val

# Define inverse min-max scaler
def inverse_minmax_scaler(data_scaled, min_val, max_val):
    return data_scaled * (max_val - min_val) + min_val

# Load data
# X_train, X_test, y_train, y_test = load_data()
X = df[used_cols].copy()
y = df[['quantity_received']].copy()
X_train, X_test = train_test_split(X, test_size=0.3, random_state=42)
y_train, y_test = train_test_split(y, test_size=0.3, random_state=42)


# Quantile Transform
scaler_s = preprocessing.QuantileTransformer()
X_train[['quantity_ordered', 'quantity_submitted', 'visibility']] = \
    scaler_s.fit_transform(X_train[['quantity_ordered', 'quantity_submitted', 'visibility']])

X_test[['quantity_ordered', 'quantity_submitted', 'visibility']] = \
    scaler_s.transform(X_test[['quantity_ordered', 'quantity_submitted', 'visibility']])

# Scale target variable
y_train_scaled, y_min, y_max = minmax_scaler(y_train)
y_test_scaled, _, _ = minmax_scaler(y_test, y_min, y_max)

from catboost import CatBoostRegressor

# Initialize the CatBoost Regressor with the desired parameters
model = CatBoostRegressor(iterations=500,
                          learning_rate=0.001,
                          depth=10,
                          l2_leaf_reg=3,
                          random_seed=42,
                          loss_function='MAE',
                          od_type='Iter',
                          od_wait=20,
                          bagging_temperature=0.8,
                          verbose=100)

# Fit the model on the training data
# model.fit(X_train, y_train,
#           eval_set=(X_test, y_test),
#           early_stopping_rounds=100,
#           use_best_model=True)
model.fit(X_train, y_train_scaled,
          eval_set=(X_test, y_test_scaled),
          early_stopping_rounds=100,
          use_best_model=True)

# Evaluate model on test set
y_pred_scaled = model.predict(X_test)
# y_pred = model.predict(X_test)
y_pred = inverse_minmax_scaler(y_pred_scaled, y_min, y_max)
r2 = r2_score(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
print('R2:', r2)
print('MAE:', mae)


0:	learn: 0.0073919	test: 0.0071756	best: 0.0071756 (0)	total: 41.1ms	remaining: 20.5s
100:	learn: 0.0070395	test: 0.0068234	best: 0.0068234 (100)	total: 3.13s	remaining: 12.4s
200:	learn: 0.0067201	test: 0.0065049	best: 0.0065049 (200)	total: 6.02s	remaining: 8.95s
300:	learn: 0.0064235	test: 0.0062102	best: 0.0062102 (300)	total: 8.25s	remaining: 5.45s
400:	learn: 0.0061670	test: 0.0059559	best: 0.0059559 (400)	total: 10.4s	remaining: 2.56s
499:	learn: 0.0059354	test: 0.0057259	best: 0.0057259 (499)	total: 12.6s	remaining: 0us

bestTest = 0.005725906162
bestIteration = 499

R2: 0.1195656161787978
MAE: 9.449361831358758


In [232]:
import numpy as np
import tensorflow as tf
from sklearn.metrics import r2_score, mean_absolute_error

# Define min-max scaler
def minmax_scaler(data, min_val=None, max_val=None):
    if min_val is None:
        min_val = np.min(data)
    if max_val is None:
        max_val = np.max(data)
    scaled = (data - min_val) / (max_val - min_val)
    return scaled, min_val, max_val

# Define inverse min-max scaler
def inverse_minmax_scaler(data_scaled, min_val, max_val):
    return data_scaled * (max_val - min_val) + min_val

# Load data
# X_train, X_test, y_train, y_test = load_data()
X = df[used_cols].copy()
y = df[['quantity_received']].copy()
X_train, X_test = train_test_split(X, test_size=0.3, random_state=42, shuffle=True)
y_train, y_test = train_test_split(y, test_size=0.3, random_state=42, shuffle=True)


# Quantile Transform
scaler_s = preprocessing.QuantileTransformer()
X_train[['quantity_ordered', 'quantity_submitted', 'visibility']] = \
    scaler_s.fit_transform(X_train[['quantity_ordered', 'quantity_submitted', 'visibility']])

X_test[['quantity_ordered', 'quantity_submitted', 'visibility']] = \
    scaler_s.transform(X_test[['quantity_ordered', 'quantity_submitted', 'visibility']])

# Scale target variable
y_train_scaled, y_min, y_max = minmax_scaler(y_train)
y_test_scaled, _, _ = minmax_scaler(y_test, y_min, y_max)

# Define model
model = tf.keras.Sequential([
  tf.keras.layers.Dense(64, activation='relu', input_shape=(len(used_cols),)),
  tf.keras.layers.Dense(32, activation='relu'),
  tf.keras.layers.Dense(1)
])

# Compile model
optimizer = tf.keras.optimizers.Adam(0.0001)

model.compile(optimizer=optimizer,
              loss='mae',
              metrics=['mae', 'mse'])

# Train model
history = model.fit(X_train, y_train_scaled, epochs=10, batch_size=12,
                    validation_split=0.2)

# Evaluate model on test set
y_pred_scaled = model.predict(X_test)
y_pred = inverse_minmax_scaler(y_pred_scaled, y_min, y_max)
r2 = r2_score(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
print('R2:', r2)
print('MAE:', mae)


Epoch 1/10
3533/3533 [==============================] - 10s 2ms/step - loss: 0.0195 - mae: 0.0195 - mse: 0.0017 - val_loss: 0.0090 - val_mae: 0.0090 - val_mse: 5.2333e-04
Epoch 2/10
3533/3533 [==============================] - 9s 2ms/step - loss: 0.0074 - mae: 0.0074 - mse: 2.8831e-04 - val_loss: 0.0072 - val_mae: 0.0072 - val_mse: 4.5448e-04
Epoch 3/10
3533/3533 [==============================] - 8s 2ms/step - loss: 0.0063 - mae: 0.0063 - mse: 2.4856e-04 - val_loss: 0.0064 - val_mae: 0.0064 - val_mse: 4.2371e-04
Epoch 4/10
3533/3533 [==============================] - 8s 2ms/step - loss: 0.0059 - mae: 0.0059 - mse: 2.3228e-04 - val_loss: 0.0059 - val_mae: 0.0059 - val_mse: 4.0303e-04
Epoch 5/10
3533/3533 [==============================] - 8s 2ms/step - loss: 0.0056 - mae: 0.0056 - mse: 2.2336e-04 - val_loss: 0.0055 - val_mae: 0.0055 - val_mse: 4.0115e-04
Epoch 6/10
3533/3533 [==============================] - 7s 2ms/step - loss: 0.0055 - mae: 0.0055 - mse: 2.1740e-04 - val_loss: 0.0055

In [200]:


import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.metrics import mean_absolute_error, r2_score


# X_train, X_test, y_train, y_test = load_data()
X = df[used_cols].copy()
y = df[['quantity_received']].copy()
X_train, X_test = train_test_split(X, test_size=0.3, random_state=42, shuffle=True)
y_train, y_test = train_test_split(y, test_size=0.3, random_state=42, shuffle=True)

# Scale target variable
y_train_scaled, y_min, y_max = minmax_scaler(y_train)
y_test_scaled, _, _ = minmax_scaler(y_test, y_min, y_max)



# Quantile Transform
scaler_s = preprocessing.QuantileTransformer()
X_train[['quantity_ordered', 'quantity_submitted', 'visibility']] = \
    scaler_s.fit_transform(X_train[['quantity_ordered', 'quantity_submitted', 'visibility']])

X_test[['quantity_ordered', 'quantity_submitted', 'visibility']] = \
    scaler_s.transform(X_test[['quantity_ordered', 'quantity_submitted', 'visibility']])


# Define the model architecture
inputs = keras.Input(shape=(X_train.shape[1],))
x = layers.Dense(128, activation='relu')(inputs)
x = layers.Dense(64, activation='relu')(x)
outputs = layers.Dense(1)(x)
model = keras.Model(inputs, outputs)

# Compile the model
model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.001),
              loss='mean_absolute_error',
              metrics=[keras.metrics.MeanAbsoluteError(), keras.metrics.RootMeanSquaredError()])

# Train the model
history = model.fit(X_train, y_train_scaled,
                    epochs=100,
                    batch_size=32,
                    validation_data=(X_test, y_test_scaled),
                    verbose=0)

# Evaluate model on test set
y_pred_scaled = model.predict(X_test)
y_pred = inverse_minmax_scaler(y_pred_scaled, y_min, y_max)
r2 = r2_score(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
print('R2:', r2)
print('MAE:', mae)

710/710 [==============================] - 1s 1ms/step
R2: 0.2905953248497468
MAE: 6.763235931276072


In [173]:
# # Define min-max scaler
# def minmax_scaler(data, min_val=None, max_val=None):
#     if min_val is None:
#         min_val = np.min(data)
#     if max_val is None:
#         max_val = np.max(data)
#     scaled = (data - min_val) / (max_val - min_val)
#     return scaled, min_val, max_val

# # Define inverse min-max scaler
# def inverse_minmax_scaler(data_scaled, min_val, max_val):
#     return data_scaled * (max_val - min_val) + min_val

In [194]:
# Evaluate model on test set

df['quantity_received'], _, _ = minmax_scaler(df['quantity_received'], y_min, y_max)
df['preds'] = model.predict(df[used_cols])
df['preds'] = inverse_minmax_scaler(df['preds'], y_min, y_max)
df['quantity_received'] = inverse_minmax_scaler(df['quantity_received'], y_min, y_max)



2366/2366 [==============================] - 4s 2ms/step


In [195]:
df[['quantity_received', 'preds']]

,quantity_received,preds
0,0.004848,7.869305
1,0.009697,17.458210
2,0.000000,1.790797
3,0.021818,40.308163
4,0.014545,26.575327
...,...,...
75691,0.003636,10.424611
75692,0.058182,24.796078
75693,0.003636,11.767908
75694,0.021818,21.281513


In [196]:
new_metric(df)

TypeError: 'numpy.float64' object is not callable

In [132]:
loss

[0.004326103255152702, 0.004326103255152702, 0.00018384327995590866]

In [133]:
# Performance metrics by epoch
hist = pd.DataFrame(history.history)
hist['epoch'] = history.epoch
hist

,loss,mae,mse,val_loss,val_mae,val_mse,epoch
0,0.009169,0.009169,0.000449,0.005723,0.005723,0.000227,0
1,0.005436,0.005436,0.000269,0.005051,0.005051,0.000208,1
2,0.005009,0.005009,0.000255,0.004790,0.004790,0.000202,2
3,0.004801,0.004801,0.000248,0.004668,0.004668,0.000193,3
4,0.004674,0.004674,0.000246,0.004548,0.004548,0.000194,4
5,0.004593,0.004593,0.000243,0.004742,0.004742,0.000191,5
6,0.004561,0.004561,0.000244,0.004680,0.004680,0.000192,6
7,0.004505,0.004505,0.000242,0.004409,0.004409,0.000190,7
8,0.004440,0.004440,0.000242,0.004337,0.004337,0.000185,8
9,0.004386,0.004386,0.000239,0.004326,0.004326,0.000184,9


In [125]:
df['preds'] = model.predict(df[req_cols])

2366/2366 [==============================] - 3s 1ms/step


In [136]:
y_pred = model.predict(x_test)
# Scale data back
y_test_inv = inverse_minmax_scaler(y_test, np.min(y), np.max(y))
y_pred_inv = inverse_minmax_scaler(y_pred, np.min(y), np.max(y)) 

print('R2:', r2_score(y_test_inv, y_pred_inv))
print('MAE:', mean_absolute_error(y_test_inv, y_pred_inv))

710/710 [==============================] - 2s 2ms/step
R2: 0.3517406190505866
MAE: 7.138067550187244


In [ ]:
y_pred = model.predict(x_test)
# Scale data back
y_test_inv = inverse_minmax_scaler(y_test, np.min(y), np.max(y))
y_pred_inv = inverse_minmax_scaler(y_pred, np.min(y), np.max(y)) 

print('R2:', r2_score(y_test_inv, y_pred_inv))
print('MAE:', mean_absolute_error(y_test_inv, y_pred_inv))

new_metric(df)

In [86]:

# df_unique['quantity_received'] = scaler_t.fit_transform(df_unique['quantity_received'].values.reshape(-1, 1))
# df_test['quantity_received'] = scaler_t.transform(df_test['quantity_received'].values.reshape(-1, 1))



# Inverse transform the standardized data
# scaler_t.inverse_transform(df_test['quantity_received'].values.reshape(1, -1))
# y_pred = df_test['preds'].copy()

# # Scale data back
# y_test_inv = y_test*y_std_t + y_mean_t
# y_pred_inv = y_pred*y_std_t + y_mean_t

In [87]:

# df_test['preds'] = y_pred_inv

In [88]:
# y_test_inv

In [89]:
# y_pred_inv

In [90]:
# new_metric(df_test)

In [91]:

# print('R2:', r2_score(y_test_inv, y_pred_inv))
# print('MAE:', mean_absolute_error(y_test_inv, y_pred_inv))

In [118]:
# Scale data back
# y_inverse = scaler.inverse_transform(y_scaled)

# y_test_inv = scaler_t.inverse_transform(df['quantity_received'].values.reshape(1, -1))
# y_pred_inv = scaler_t.inverse_transform(df['preds'].values.reshape(1, -1))

y_test_inv = df['quantity_received'].copy()
y_pred_inv = df['preds'].copy()

print('R2:', r2_score(y_test_inv, y_pred_inv))
print('MAE:', mean_absolute_error(y_test_inv, y_pred_inv))

R2: 0.2799510671224511
MAE: 0.005173128853860543


In [119]:
y_test_inv

0        0.004848
1        0.009697
2        0.000000
3        0.021818
4        0.014545
           ...   
75691    0.003636
75692    0.058182
75693    0.003636
75694    0.021818
75695    0.000000
Name: quantity_received, Length: 75696, dtype: float64

In [120]:
y_test_inv

0        0.004848
1        0.009697
2        0.000000
3        0.021818
4        0.014545
           ...   
75691    0.003636
75692    0.058182
75693    0.003636
75694    0.021818
75695    0.000000
Name: quantity_received, Length: 75696, dtype: float64

0.7234467558380431

In [117]:
import tensorflow as tf

# define and train your model here

# save the model
model.save('Final_TENSORFLOW_MODEL')

INFO:tensorflow:Assets written to: Final_TENSORFLOW_MODEL\assets


INFO:tensorflow:Assets written to: Final_TENSORFLOW_MODEL\assets


In [118]:
model2 = tf.keras.models.load_model('Final_TENSORFLOW_MODEL')

In [135]:
df_test['preds'] = model2.predict(df_test[req_cols])
new_metric(df_test)

KeyError: "['d_id_440', 'd_id_79', 'd_id_216', 'd_id_462', 'd_id_746', 'd_id_210', 'quantity_received_s', 'quantity_ordered_s', 'quantity_submitted_s'] not in index"

In [120]:
df_test.shape

(22406, 85)

In [ ]:
df_test.to_csv('final_scored_test_file.csv', index = False)